In [1]:
import os
import time
import yaml
import math
import pandas as pd
import numpy as np

In [2]:
from data.excel2pd import data_load
from data.preprocessing import data_preprocessing
from data.train_test_split import data2Window, X_Y2train_test, X_Y_from_data
from data.CustomDataset import CustomDataset

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset # 텐서데이터셋
from torch.utils.data import DataLoader # 데이터로더
from model.model import Transformer

import sys
sys.setrecursionlimit(4000)

In [4]:
# torch version, gpu 사용 여부, 이름 체크
print("torch version>>", torch.__version__)
print("GPU 사용 유무>>", torch.cuda.is_available())
if torch.cuda.is_available() :
    print("GPU 이름>> ", torch.cuda.get_device_name(0))
    device = "cuda:0" 
else :
    device = "cpu"
print(device)

torch version>> 1.13.1
GPU 사용 유무>> True
GPU 이름>>  NVIDIA GeForce RTX 2080
cuda:0


In [5]:
f = open("config.yaml", 'r')
config = yaml.load(f, Loader=yaml.FullLoader)

data = data_load(11)
scaler, data = data_preprocessing(data, config['data_options'])

X, Y = X_Y_from_data(data)

X_train, Y_train, X_test, Y_test = X_Y2train_test(X=X, Y=Y,
                                                  output_window=config['output_window'],
                                                  input_size=config['data_options']['feature'],
                                                  stride=config['stride'])

print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)

(51166, 6, 1) (51166, 6)
(309, 6) (309, 6)


In [6]:
dataset = CustomDataset(X_train, Y_train)
dataloader = DataLoader(dataset, batch_size = config['batch'], shuffle=False)

In [12]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)